In [ ]:
!git clone https://github.com/Mike030668/Project_glass.git -q

In [ ]:
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  import os
  os.chdir('/content/Project_glass')
  os.getcwd()

else:
  print('Not running on CoLab')

Running on CoLab


In [ ]:
from prepare_data.constants import GLASS_COLS, FUTURE, MAIN_COLS, DEPTH, PREDICT_LAG, BATCH_SIZE, MAKE_LOG_VOL
from prepare_data.const_predprice import  TARGET_COLS, BASE_COLS, MAKE_LOG_TARGET
from prepare_data.utils import future_sequence, split_sequence, clean_dataset
import tensorflow as tf  # библиотека машинного обучения
import pandas as pd # библиотека пандас
import numpy as np # библиотека нампи
import joblib
from tqdm.notebook import tqdm   # отрисовка прохождения цикла
import warnings # библиотека сообщений по ошибкам
warnings.filterwarnings("ignore") # игнорировать сообщения ошибок

## Готовим производные данных

In [ ]:
all_data = pd.read_csv("/content/Project_glass/stock_data/BTCUSDT_1 _min.csv",
                       index_col=0, parse_dates=True )
print(f'Размер данных {all_data.shape}')

Размер данных (8317, 54)


In [ ]:
# колонки для тренировочной выборки
TREND_PRED_COLS = MAIN_COLS + GLASS_COLS

# колонки для тренировочной выборки
PRICE_PRED_COLS = BASE_COLS + GLASS_COLS

GLASS = len(GLASS_COLS)

In [ ]:
trend_model =  tf.keras.models.load_model('./predtrend/model')
price_model =  tf.keras.models.load_model('./predprice/model')

x_trend_main_scr = joblib.load("/content/Project_glass/predtrend/x_main_scailer.save")
x_trend_glass_scr = joblib.load("/content/Project_glass/predtrend/x_glass_scailer.save")
trend_scalers = [x_trend_main_scr, x_trend_glass_scr]

x_price_main_scr = joblib.load("/content/Project_glass/predprice/x_main_scailer.save")
x_price_glass_scr = joblib.load("/content/Project_glass/predprice/x_glass_scailer.save")
y_price_scr = joblib.load("/content/Project_glass/predprice/y_scailer.save")
price_scalers = [x_price_main_scr, x_price_glass_scr, y_price_scr]

In [ ]:
import random
def prepare_data(df_in: pd.DataFrame,
                        depth:int,
                        make_log,
                        scalers,
                        glass,
                        features
                      ):
    # обогащаем данные ранее созданной функцией
    ds = df_in.copy()
    if make_log:
       ds["Volume"] = ds["Volume"].apply(lambda x: np.log(x))
    # Удаление строк с неполными данными
    ds = clean_dataset(ds)
    # оставляем на глубину depth и только нужные колонки
    x_data = ds[features][-depth:].values
    # нормализуем
    x_out = np.zeros_like(x_data)
    x_out[:,:-glass] = scalers[0].transform(x_data[:,:-glass])  # трансформируем x_train
    x_out[:,-glass:] = scalers[1].transform(x_data[:,-glass:])  # трансформируем x_train

    # выводим с добавлением измерения по axis=0
    return np.expand_dims(x_out, axis=0)

def tresh_trend(trend, trsh_d, trsh_u):
    out = np.zeros(trend.shape[0])
    out[trend > trsh_u] = 1
    out[trend < trsh_d] = -1
    return out.astype(int)

#######################################################################################
def cond_long (state,
               id_pr_up,
               id_tr_up,
               id_pr_lw,
               id_tr_lw,
               ):

    cur_price = state["last_price"]
    # Вход лонг
    # Покупаем когда текущая позиция 0 (ничего не куплено) И
    # это в тестировщике

    # Сетка 1 показывает id_pr_up предсказаний выше текущей цены И
    cond_1 =  all(state["pred_prices"][id_pr_up] > cur_price)

    # Сетка 2 изменила состояние Флэт или Падение
    cond_2 = all(state["pred_trend"][-2] < 1)

    # на Рост по всем id_tr_up.
    cond_3 = all(state["pred_trend"][-1][id_tr_up] == 1)
    cond_4 = cond_2 and cond_3

    # Вход лонг
    enter_long = cond_1 and cond_4


    # Выход из лонга
    # Продаем позицию если
    # Сетка 1 показывает id_pr_lw предсказания ниже текущей цены И
    cond_5 =  all(state["pred_prices"][id_pr_lw] < cur_price)

    # Сетка 2 изменила состояние на Падение на id_pr_lw минутах.
    a = np.array(state["pred_trend"][-2][id_pr_lw])
    cond_6 = all(a >= 0)
    a = np.array(state["pred_trend"][-1][id_pr_lw])
    cond_7 = all(a == -1)

    # Выход из лонга
    exit_long = cond_5 and cond_6 and cond_7


    # Вход лонг
    if enter_long and not exit_long: return 1
    # Выход из лонга
    elif  not enter_long and exit_long: return -1
    # спим
    else:  return 0


def cond_short(state,
               id_pr_up,
               id_tr_up,
               id_pr_lw,
               id_tr_lw,
                ):

    cur_price = state["last_price"]
    # Вход шорт
    # Покупаем когда текущая позиция 0 (ничего не продано) И
    # это в тестировщике
    # Сетка 1 показывает id_pr_lw предсказаний ниже текущей цены И
    cond_1 =  all(state["pred_prices"][id_pr_lw] + delta(pred[0] - pred_pred[0]) < cur_price)

    # Сетка 2 изменила состояние Флэт или Рост
    cond_2 = all(state["pred_trend"][-2][id_tr_lw] > -1)
    # Падение по всем пяти минутам.
    cond_3 = all(state["pred_trend"][-1][id_tr_lw] == -1)
    cond_4 = cond_2 and cond_3
    # Вход лонг
    enter_short = cond_1 and cond_4

    # Выход из шорта
    # Продаем позицию если
    # Сетка 1 показывает id_pr_up предсказания выше текущей цены  И
    cond_5 =  all(state["pred_prices"][id_pr_up] > cur_price)
    # Сетка 2 изменила состояние на Рост на  id_tr_up минуте
    a = np.array(state["pred_trend"][-2][id_tr_up])
    cond_6 = all(a <= 0)
    a = np.array(state["pred_trend"][-1][id_tr_up])
    cond_7 = all(a == 1)
    # Выход из шорта
    exit_short = cond_5 and cond_6 and cond_7

    # Вход шорта
    if enter_short and not exit_short: return -1
    # Выход из шорта
    elif  not enter_short and exit_short: return 1
    # спим
    else:  return 0
#######################################################################################

def long_Short(action,
               lg_id_pr_up,
               lg_id_tr_up,
               lg_id_pr_lw,
               lg_id_tr_lw,
               sh_id_pr_up,
               sh_id_tr_up,
               sh_id_pr_lw,
               sh_id_tr_lw,
 ):
    """
    action in ('long', 'short', 'mix')
    """
    def mix_action (state, long_action, short_action):
      short_act = short_action(state, sh_id_pr_up, sh_id_tr_up, sh_id_pr_lw, sh_id_tr_lw)
      long_act = long_action(state, lg_id_pr_up, lg_id_tr_up, lg_id_pr_lw, lg_id_tr_lw)
      if action == 'long' : return [long_act, 0]
      elif action == 'short' : return [0, short_act]
      else:
          return  [long_act, short_act]
    return mix_action



def making_signals(past_df: pd.DataFrame,
                    check_df: pd.DataFrame,
                    pred_lag: int,
                    glass: int,
                    comis:float,
                    fun_action: object,
                    model_price: object,
                    model_trend: object,
                    scalers_price: object,
                    scalers_trend: object,
                    features_price: list,
                    features_trend: list,
                    use_force_action = False,
                    show_unique_signals = True,
                    ):

    """
    Args:
        past_df (pd.DataFrame) - прошлые OHCL данные до тестируемых
        check_df (pd.DataFrame) - тестируемые OHCL данные
        pred_lag: int - шаг предсказания
        model (class) - ранее обученная модель
        use_force_action (bool): False - выводим чистые действия,
                                 True - выводим действия с учетом уверенности,
        show_unique_signals (bool) - показать ли состав чистых действий

    Return:
         df_signal(pd.DataFrame) - копия OHCL датафрейма из check_df с колонкой Signal
    """
    # сбор действий
    all_actions = [[0, 0]]
    # сбор последнего активного действия и цены
    last_state = {
        "last_price" : 0,
        "pred_prices" : [[]],
        "pred_trend" : [[]]
                    }

    # итеррируемся по длине изучаемого датафрейма
    # берем на страте прошлые данные
    for i in tqdm(range(check_df.shape[0]-pred_lag), unit ="step",
                      desc ="Пробегаемся по всем отсчетам"):
        if not i: # берем на страте прошлые данные
            check_data = past_df.copy()
        # далее отшипываем вначале 1 свечу
        else: #  и присоединяем в коней новую из изучаемого датафрейма
            check_data = check_data[1:].append(check_df[i:i+1])

        # обогащаем данные по аналогии как готовили для убучени
        to_pred_price = prepare_data(df_in = check_data,
                                      depth = DEPTH ,    # ранее заданная глубина сбора данных в прошлое
                                      make_log = MAKE_LOG_VOL,
                                      scalers = scalers_price,
                                      glass = glass,
                                      features = features_price
                                      )
        # предсказание модели
        pred_price = model_price.predict(to_pred_price, verbose=False)
        pred_price = scalers_price[2].inverse_transform(pred_price)

        if MAKE_LOG_TARGET: pred_price = np.exp(pred_price)

        last_state["pred_prices"] = pred_price[0].astype(float)

         # обогащаем данные по аналогии как готовили для убучени
        to_pred_trend = prepare_data(df_in = check_data,
                                      depth = DEPTH ,    # ранее заданная глубина сбора данных в прошлое
                                      make_log = MAKE_LOG_VOL,
                                      scalers = scalers_trend,
                                      glass = GLASS,
                                      features = features_trend
                                      )
        # предсказание модели
        pred_trend = model_trend.predict(to_pred_trend, verbose=False)[0].astype(float)
        pred_trend = tresh_trend(pred_trend, trsh_d = -0.1, trsh_u = 0.1)
        last_state["pred_trend"].append(pred_trend)
        price = check_data.Close[-1]
        last_state["last_price"] = price

        if i:
          last_state["pred_trend"] = last_state["pred_trend"][-2:]
          to_action = fun_action(last_state, cond_long, cond_short)
          all_actions.append(to_action)

    # переводим классы сигналов в массив сигналов
    all_actions = np.vstack(all_actions)
    # смещаем на pred_lag
    df_signal = check_df[pred_lag:].copy()
    df_signal['to_long']  = all_actions[:, 0]
    df_signal['to_short'] = all_actions[:, 1]
    # берем чистые действия действия
    return  df_signal


In [29]:
COMIS = 0.0002

FUTURE = 3000


# длина взгляда назад
BACK = DEPTH + PREDICT_LAG
TEST_SIZE = 5000
# данные назад
past_data =  all_data[FUTURE-BACK : FUTURE - PREDICT_LAG].copy()

# тестируемы датафрейм
test_data = all_data[FUTURE-PREDICT_LAG : FUTURE + TEST_SIZE].copy()

# получаем датафрейм с сигналом
df_test_signal = making_signals(past_data,
                                test_data,
                                comis = COMIS,
                                pred_lag = PREDICT_LAG,
                                glass = GLASS,

                                fun_action = long_Short(
                                action = "mix",      # mix = long_Short
                                # long индексы [0,1,2,3,4]
                                lg_id_pr_up = [0,1], # id предсказаний выше текущей цены
                                lg_id_tr_up = [0,1], # Рост по id
                                lg_id_pr_lw = [0], # id предсказаний ниже текущей цены
                                lg_id_tr_lw = [0], # Падение по id
                                # short индексы [0,1,2,3,4]
                                sh_id_pr_up = [0,1], # id предсказаний выше текущей цены
                                sh_id_tr_up = [0,1], # Рост по id
                                sh_id_pr_lw = [0], # id предсказаний ниже текущей цены
                                sh_id_tr_lw = [0], # Падение по id
                                ),

                                model_price = price_model,
                                model_trend = trend_model,
                                scalers_price = price_scalers,
                                scalers_trend = trend_scalers,
                                features_price = PRICE_PRED_COLS,
                                features_trend = TREND_PRED_COLS,
                                #use_force_action = False,
                                show_unique_signals = True
                                )

Пробегаемся по всем отсчетам:   0%|          | 0/5000 [00:00<?, ?step/s]

# Тестирование на основе  Backtest, Strategy

In [ ]:
!pip install backtesting -q
# Библиотеки backtesting
from backtesting import Backtest, Strategy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [30]:
class Long_Strategy(Strategy):

    def init(self):
      #self.column = use_price
      self.to_long = self.I(lambda x: x, self.data.to_long, name ='to_long')

    def next(self):
      # торгуем по крайней цене закрытия
      price = self.data.Close[-1]
      # определяем размер/силу действия из сигнали
      size = abs(self.to_long[-1])

      # закрытие лонга
      if (self.position.is_long and 0> self.to_long[-1] >= -1):
          # Текущий капитал (денежные средства плюс активы).
          print('Текущий капитал ', round(self.equity))
          print('______________________________________________')
          print('Закрытие позиции')
          # Размер позиции в единицах актива. Отрицательно, если позиция короткая.
          print('Размер позиции', self.position.size)
          self.position.close()
          #print('trades ', self.trades)
          # Прибыль (положительная) или убыток (отрицательная) текущей позиции в денежных единицах.
          print('Прибыль/убыток (позиции в деньгах', self.position.pl)
          # Прибыль (положительная) или убыток (отрицательная) текущей позиции в процентах.
          print('Прибыль/убыток позиции в %', self.position.pl_pct)
          # True, если позиция длинная (размер позиции положительный).
          print('position.is_long', self.position.is_long)
          # True, если позиция короткая (размер позиции отрицательный).
          print('position.is_short', self.position.is_short)
          print('______________________________________________')
          print()


      # Long entry
      elif (self.position.size == 0 and 0 < self.to_long[-1] <= 1):
          # Текущий капитал (денежные средства плюс активы).
          print('Текущий капитал ', round(self.equity))
          print('______________________________________________')
          print('вход в покупку')
          self.buy()
          #self.position.entry_price = price*size
          print('orders ', self.orders)
          print('______________________________________________')
          print()


class Short_Strategy(Strategy):

    def init(self):
      self.to_short = self.I(lambda x: x, self.data.to_short, name='to_short')

    def next(self):
      # торгуем по крайней цене закрытия
      price = self.data.Close[-1]
      # определяем размер/силу действия из сигнали
      size = abs(self.to_short[-1])

      # закрытие шорта
      if (self.position.is_short and 0< self.to_short[-1] <= 1):
        # Текущий капитал (денежные средства плюс активы).
        print('Текущий капитал ', round(self.equity))
        print('______________________________________________')
        print('Закрытие позиции')
        # Размер позиции в единицах актива. Отрицательно, если позиция короткая.
        print('Размер позиции', self.position.size)
        self.position.close()
        # Прибыль (положительная) или убыток (отрицательная) текущей позиции в денежных единицах.
        print('Прибыль/убыток (позиции в деньгах', self.position.pl)
        # Прибыль (положительная) или убыток (отрицательная) текущей позиции в процентах.
        print('Прибыль/убыток позиции в %', self.position.pl_pct)
        # True, если позиция длинная (размер позиции положительный).
        print('position.is_long', self.position.is_long)
        # True, если позиция короткая (размер позиции отрицательный).
        print('position.is_short', self.position.is_short)
        print('______________________________________________')
        print()

      # Short entry
      if (self.position.size == 0 and  0> self.to_short[-1] >= -1):
          # Текущий капитал (денежные средства плюс активы).
          print('Текущий капитал ', round(self.equity))
          print('______________________________________________')
          print('вход в продажу')
          self.sell()
          # Прибыль (положительная) или убыток (отрицательная) текущей позиции в денежных единицах.
          print('Прибыль/убыток (позиции в деньгах', self.position.pl)
          # Прибыль (положительная) или убыток (отрицательная) текущей позиции в процентах.
          print('Прибыль/убыток позиции в %', self.position.pl_pct)
          # True, если позиция длинная (размер позиции положительный).
          print('position.is_long', self.position.is_long)
          # True, если позиция короткая (размер позиции отрицательный).
          print('position.is_short', self.position.is_short)
          print('______________________________________________')
          print()


class Long_n_Short_Strategy(Strategy):

    def init(self):
      self.to_long = self.I(lambda x: x, self.data.to_long, name='to_long')
      self.to_short = self.I(lambda x: x, self.data.to_short, name='to_short')

    def next(self):
      # торгуем по крайней цене закрытия
      price = self.data.Close[-1]

      # закрытие лонга
      if (self.position.is_long and 0 > self.to_long[-1] >= -1):
          # Текущий капитал (денежные средства плюс активы).
          print('Текущий капитал ', round(self.equity))
          print('______________________________________________')
          print('Закрытие позиции LONG')
          # Размер позиции в единицах актива. Отрицательно, если позиция короткая.
          print('Размер позиции', self.position.size)
          self.position.close()
          #print('trades ', self.trades)
          # Прибыль (положительная) или убыток (отрицательная) текущей позиции в денежных единицах.
          print('Прибыль/убыток (позиции в деньгах', self.position.pl)
          # Прибыль (положительная) или убыток (отрицательная) текущей позиции в процентах.
          print('Прибыль/убыток позиции в %', self.position.pl_pct)
          # True, если позиция длинная (размер позиции положительный).
          print('position.is_long', self.position.is_long)
          # True, если позиция короткая (размер позиции отрицательный).
          print('position.is_short', self.position.is_short)
          print('______________________________________________')
          print()


      # закрытие шорта
      elif (self.position.is_short and 0 < self.to_short[-1] <= 1):
        # Текущий капитал (денежные средства плюс активы).
        print('Текущий капитал ', round(self.equity))
        print('______________________________________________')
        print('Закрытие позиции')
        # Размер позиции в единицах актива. Отрицательно, если позиция короткая.
        print('Размер позиции', self.position.size)
        self.position.close()
        # Прибыль (положительная) или убыток (отрицательная) текущей позиции в денежных единицах.
        print('Прибыль/убыток (позиции в деньгах', self.position.pl)
        # Прибыль (положительная) или убыток (отрицательная) текущей позиции в процентах.
        print('Прибыль/убыток позиции в %', self.position.pl_pct)
        # True, если позиция длинная (размер позиции положительный).
        print('position.is_long', self.position.is_long)
        # True, если позиция короткая (размер позиции отрицательный).
        print('position.is_short', self.position.is_short)
        print('______________________________________________')
        print()


      # Long entry
      if (self.position.size == 0 and 0 < self.to_long[-1] <= 1):
          # Текущий капитал (денежные средства плюс активы).
          print('Текущий капитал ', round(self.equity))
          print('______________________________________________')
          print('вход в покупку')
          self.buy()
          #self.position.entry_price = price*size
          print('orders ', self.orders)
          print('______________________________________________')
          print()

      # Short entry
      elif (self.position.size == 0 and  0> self.to_short[-1] >= -1):
          # Текущий капитал (денежные средства плюс активы).
          print('Текущий капитал ', round(self.equity))
          print('______________________________________________')
          print('вход в продажу')
          self.sell()
          print('orders ', self.orders)
          print('______________________________________________')
          print()

      # Проверка пересечения условий
      if (self.position.size == 0 and 0 < self.to_long[-1] <= 1) and \
         (self.position.size == 0 and  0> self.to_short[-1] >= -1):
         print("Пересечение Long entry и Short entry")


class Reverse_Strategy(Strategy):

      def init(self):
          self.to_long = self.I(lambda x: x, self.data.to_long, name='to_long')
          self.to_short = self.I(lambda x: x, self.data.to_short, name='to_short')

      def next(self):
          # торгуем по крайней цене закрытия
          price = self.data.Close[-1]
          # определяем размер/силу действия из сигнали
          #size = abs(self.to_long[-1])
          size = 0.1*self.equity

          # Long entry
          if (self.position.size == 0  and 0 < self.to_long[-1] <= 1):
              # Текущий капитал (денежные средства плюс активы).
              print('Текущий капитал ', round(self.equity))
              print('______________________________________________')
              print('вход в Long')
              self.buy(size = size)
              #self.position.entry_price = price*size
              print('orders ', self.orders)
              print('______________________________________________')
              print()

          # Short entry
          elif (self.position.size == 0 and  0> self.to_short[-1] >= -1):
              # Текущий капитал (денежные средства плюс активы).
              print('Текущий капитал ', round(self.equity))
              print('______________________________________________')
              print('вход в Short')
              self.sell(size = size)
              #self.position.entry_price = price*size
              print('orders ', self.orders)
              print('______________________________________________')
              print()

          # закрытие лонга
          if (self.position.is_long and 0 > self.to_short[-1] >= -1):
              # Текущий капитал (денежные средства плюс активы).
              print('Текущий капитал ', round(self.equity))
              print('______________________________________________')
              print('Закрытие позиции LONG')
              # Размер позиции в единицах актива. Отрицательно, если позиция короткая.
              print('Размер позиции', self.position.size)
              self.position.close()
              print('______________________________________________')
              print('вход в продажу')
              self.sell(size = size)
              #print('trades ', self.trades)
              # Прибыль (положительная) или убыток (отрицательная) текущей позиции в денежных единицах.
              print('Прибыль/убыток (позиции в деньгах', self.position.pl)
              # Прибыль (положительная) или убыток (отрицательная) текущей позиции в процентах.
              print('Прибыль/убыток позиции в %', self.position.pl_pct)
              # True, если позиция длинная (размер позиции положительный).
              print('position.is_long', self.position.is_long)
              # True, если позиция короткая (размер позиции отрицательный).
              print('position.is_short', self.position.is_short)
              print('______________________________________________')
              print()

          # закрытие шорта
          if (self.position.is_short and 0< self.to_long[-1] <= 1):
            # Текущий капитал (денежные средства плюс активы).
            print('Текущий капитал ', round(self.equity))
            print('______________________________________________')
            print('Закрытие позиции Short')
            # Размер позиции в единицах актива. Отрицательно, если позиция короткая.
            print('Размер позиции', self.position.size)
            self.position.close()
            print('______________________________________________')
            print('вход в Long')
            self.buy(size = size)
            # Прибыль (положительная) или убыток (отрицательная) текущей позиции в денежных единицах.
            print('Прибыль/убыток (позиции в деньгах', self.position.pl)
            # Прибыль (положительная) или убыток (отрицательная) текущей позиции в процентах.
            print('Прибыль/убыток позиции в %', self.position.pl_pct)
            # True, если позиция длинная (размер позиции положительный).
            print('position.is_long', self.position.is_long)
            # True, если позиция короткая (размер позиции отрицательный).
            print('position.is_short', self.position.is_short)
            print('______________________________________________')
            print()

In [31]:
CAPITAL = 100000

# Стратегия
bt = Backtest(df_test_signal, Short_Strategy, cash= CAPITAL, commission=COMIS,
              hedging = True, trade_on_close=False,  exclusive_orders=True)
stats = bt.run()
bt.plot(plot_volume=True, relative_equity=True)
stats

Текущий капитал  100000
______________________________________________
вход в продажу
Прибыль/убыток (позиции в деньгах 0
Прибыль/убыток позиции в % 0.0
position.is_long False
position.is_short False
______________________________________________

Текущий капитал  100583
______________________________________________
Закрытие позиции
Размер позиции -1
Прибыль/убыток (позиции в деньгах 583.2088599999988
Прибыль/убыток позиции в % 0.011227363409693658
position.is_long False
position.is_short True
______________________________________________

Текущий капитал  100583
______________________________________________
вход в продажу
Прибыль/убыток (позиции в деньгах 0
Прибыль/убыток позиции в % 0.0
position.is_long False
position.is_short False
______________________________________________

Текущий капитал  100968
______________________________________________
Закрытие позиции
Размер позиции -1
Прибыль/убыток (позиции в деньгах 384.5676400000011
Прибыль/убыток позиции в % 0.00751819851752255

Start                     2024-02-22 19:00:00
End                       2024-02-26 06:27:00
Duration                      3 days 11:27:00
Exposure Time [%]                        7.32
Equity Final [$]                  100967.9765
Equity Peak [$]                   100967.9765
Return [%]                           0.967976
Buy & Hold Return [%]               -0.367739
Return (Ann.) [%]                   32.134006
Volatility (Ann.) [%]                4.825069
Sharpe Ratio                         6.659802
Sortino Ratio                             inf
Calmar Ratio                       107.591242
Max. Drawdown [%]                   -0.298667
Avg. Drawdown [%]                   -0.065403
Max. Drawdown Duration        0 days 21:58:00
Avg. Drawdown Duration        0 days 01:17:00
# Trades                                    2
Win Rate [%]                            100.0
Best Trade [%]                       1.122929
Worst Trade [%]                      0.752015
Avg. Trade [%]                    

In [32]:
CAPITAL = 100000

# Стратегия Long
bt = Backtest(df_test_signal, Long_Strategy, cash= CAPITAL, commission=COMIS,
              hedging = True, trade_on_close=False,  exclusive_orders=True)
stats = bt.run()
bt.plot(plot_volume=True, relative_equity=True)
stats

Текущий капитал  100000
______________________________________________
вход в покупку
orders  (<Order size=1.0, contingent=0>,)
______________________________________________



Start                     2024-02-22 19:00:00
End                       2024-02-26 06:27:00
Duration                      3 days 11:27:00
Exposure Time [%]                       66.52
Equity Final [$]                 100759.54662
Equity Peak [$]                  101236.44662
Return [%]                           0.759547
Buy & Hold Return [%]               -0.367739
Return (Ann.) [%]                   73.737249
Volatility (Ann.) [%]               15.119359
Sharpe Ratio                         4.877009
Sortino Ratio                       34.121744
Calmar Ratio                       128.429112
Max. Drawdown [%]                   -0.574147
Avg. Drawdown [%]                   -0.089316
Max. Drawdown Duration        0 days 18:06:00
Avg. Drawdown Duration        0 days 01:17:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       1.495846
Worst Trade [%]                      1.495846
Avg. Trade [%]                    

In [33]:
CAPITAL = 100000

# Стратегия Long_Short
bt = Backtest(df_test_signal, Long_n_Short_Strategy, cash= CAPITAL, commission=COMIS,
              hedging = True, trade_on_close=False,  exclusive_orders=True)
stats = bt.run()
bt.plot(plot_volume=True, relative_equity=True)
stats

Текущий капитал  100000
______________________________________________
вход в продажу
orders  (<Order size=-1.0, contingent=0>,)
______________________________________________

Текущий капитал  100583
______________________________________________
Закрытие позиции
Размер позиции -1
Прибыль/убыток (позиции в деньгах 583.2088599999988
Прибыль/убыток позиции в % 0.011227363409693658
position.is_long False
position.is_short True
______________________________________________

Текущий капитал  100583
______________________________________________
вход в продажу
orders  (<Order size=-1.0, contingent=0>,)
______________________________________________

Текущий капитал  100968
______________________________________________
Закрытие позиции
Размер позиции -1
Прибыль/убыток (позиции в деньгах 384.5676400000011
Прибыль/убыток позиции в % 0.007518198517522556
position.is_long False
position.is_short True
______________________________________________



Start                     2024-02-22 19:00:00
End                       2024-02-26 06:27:00
Duration                      3 days 11:27:00
Exposure Time [%]                        7.32
Equity Final [$]                  100967.9765
Equity Peak [$]                   100967.9765
Return [%]                           0.967976
Buy & Hold Return [%]               -0.367739
Return (Ann.) [%]                   32.134006
Volatility (Ann.) [%]                4.825069
Sharpe Ratio                         6.659802
Sortino Ratio                             inf
Calmar Ratio                       107.591242
Max. Drawdown [%]                   -0.298667
Avg. Drawdown [%]                   -0.065403
Max. Drawdown Duration        0 days 21:58:00
Avg. Drawdown Duration        0 days 01:17:00
# Trades                                    2
Win Rate [%]                            100.0
Best Trade [%]                       1.122929
Worst Trade [%]                      0.752015
Avg. Trade [%]                    

In [34]:
CAPITAL = 100000

# Стратегия Reverse
bt = Backtest(df_test_signal, Reverse_Strategy, cash= CAPITAL, commission=COMIS,
              hedging = True, trade_on_close=False,  exclusive_orders=True)
stats = bt.run()
bt.plot(plot_volume=True, relative_equity=True)
stats

Текущий капитал  100000
______________________________________________
вход в Short
orders  (<Order size=-10000.0, contingent=0>,)
______________________________________________

Текущий капитал  100000
______________________________________________
вход в Short
orders  (<Order size=-10000.0, contingent=0>,)
______________________________________________

Текущий капитал  100000
______________________________________________
вход в Short
orders  (<Order size=-10000.0, contingent=0>,)
______________________________________________

Текущий капитал  100000
______________________________________________
вход в Short
orders  (<Order size=-10000.0, contingent=0>,)
______________________________________________

Текущий капитал  100000
______________________________________________
вход в Long
orders  (<Order size=10000.0, contingent=0>,)
______________________________________________



Start                     2024-02-22 19:00:00
End                       2024-02-26 06:27:00
Duration                      3 days 11:27:00
Exposure Time [%]                         0.0
Equity Final [$]                     100000.0
Equity Peak [$]                      100000.0
Return [%]                                0.0
Buy & Hold Return [%]               -0.367739
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    